Step 1: Load data and     preview

In [25]:
import pandas as pd

df = pd.read_csv('customer_support_tickets.csv')
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head(3)

Shape: (8469, 17)
Columns: ['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0


Step 2: Normalize columns and clean messages

In [26]:

df = df.rename(columns=lambda c: c.strip().lower() if isinstance(c, str) else c)

text_candidates = ['ticket description','message','text','body','comment','ticket','description','issue']
text_col = next((c for c in text_candidates if c in df.columns), None)
if text_col is None:

    obj_cols = df.select_dtypes(include='object').columns
    text_col = obj_cols[0] if len(obj_cols) else None

print("Using text column:", text_col)


import re
def clean_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    t = text.lower()

    t = re.sub(r'[^a-z0-9\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

df['clean_message'] = df[text_col].apply(clean_text) if text_col else ""
df[['clean_message']].head(3)

Using text column: ticket description


,clean_message
0,i m having an issue with the product purchased...
1,i m having an issue with the product purchased...
2,i m facing a problem with my product purchased...


Step 3: Classify issue type and assign priority (accurate logic)

In [28]:
import re


ISSUE_KEYWORDS = {
    'PAYMENT': [
        'payment', 'card', 'transaction', 'charged', 'double charge', 'billing', 'invoice',
        'declined', 'card declined', 'upi', 'netbanking'
    ],
    'LOGIN': [
        'login', 'log in', 'signin', 'sign in', 'password', 'reset password',
        'forgot password', 'otp', '2fa', 'account locked', 'unable to login'
    ],
    'DELIVERY': [
        'delivery', 'tracking', 'track order', 'shipped', 'courier',
        'delay', 'delayed', 'arrive', 'not delivered', 'order not arrived'
    ],
    'REFUND': [
        'refund', 'reimburse', 'return', 'money back', 'refund status'
    ],
    'BUG': [
        'error', 'bug', 'crash', 'exception', 'not working', 'fail', 'failing', 'broken'
    ],
    'GENERAL': []
}


PRIORITY_KEYWORDS = {
    'P0': [
        'account hacked', 'hacked', 'data breach', 'breach', 'fraud', 'fraudulent charge',
        'payment failed multiple times', 'cannot access account', 'security issue'
    ],
    'P1': [
        'payment', 'refund', 'login', 'password', 'card', 'charged', 'declined',
        'otp', 'account locked', 'unable to login'
    ],
    'P2': [
        'delivery', 'tracking', 'delay', 'delayed', 'courier', 'not delivered',
        'order not arrived', 'reschedule delivery'
    ],
    'P3': [
        'question', 'inquiry', 'how to', 'info', 'general', 'help'
    ]
}


def compile_patterns(keywords):
    pats = []
    for kw in keywords:

        pats.append(re.compile(rf'\b{re.escape(kw)}\b'))
    return pats

ISSUE_PATTERNS = {k: compile_patterns(v) for k, v in ISSUE_KEYWORDS.items()}
PRIORITY_PATTERNS = {k: compile_patterns(v) for k, v in PRIORITY_KEYWORDS.items()}

def count_matches(text, patterns):
    if not text:
        return 0
    return sum(1 for p in patterns if p.search(text))


ISSUE_ORDER = ['PAYMENT', 'LOGIN', 'DELIVERY', 'REFUND', 'BUG', 'GENERAL']

def classify_issue(text):
    t = text.lower() if isinstance(text, str) else ""
    scores = {issue: count_matches(t, ISSUE_PATTERNS[issue]) for issue in ISSUE_PATTERNS}
    best_issue = max(scores, key=lambda k: (scores[k], -ISSUE_ORDER.index(k)))
    return best_issue if scores[best_issue] > 0 else 'GENERAL'


def assign_priority(text):
    t = text.lower() if isinstance(text, str) else ""
    if count_matches(t, PRIORITY_PATTERNS['P0']) > 0:
        return 'P0'
    if count_matches(t, PRIORITY_PATTERNS['P1']) > 0:
        return 'P1'
    if count_matches(t, PRIORITY_PATTERNS['P2']) > 0:
        return 'P2'
    if count_matches(t, PRIORITY_PATTERNS['P3']) > 0:
        return 'P3'
    issue = classify_issue(t)
    return {
        'PAYMENT': 'P1',
        'REFUND': 'P1',
        'LOGIN':  'P1',
        'DELIVERY':'P2',
        'BUG':    'P1',
        'GENERAL':'P3'
    }.get(issue, 'P3')


df['issue_type'] = df['clean_message'].apply(classify_issue)
df['priority']   = df['clean_message'].apply(assign_priority)


print("Priority distribution:\n", df['priority'].value_counts())
df[['clean_message', 'issue_type', 'priority']].head(10)

Priority distribution:
 priority
P3    6649
P1    1709
P2     105
P0       6
Name: count, dtype: int64


,clean_message,issue_type,priority
0,i m having an issue with the product purchased...,PAYMENT,P1
1,i m having an issue with the product purchased...,GENERAL,P3
2,i m facing a problem with my product purchased...,GENERAL,P3
3,i m having an issue with the product purchased...,GENERAL,P3
4,i m having an issue with the product purchased...,DELIVERY,P2
5,i m facing a problem with my product purchased...,GENERAL,P3
6,i m unable to access my product purchased acco...,LOGIN,P1
7,i m having an issue with the product purchased...,GENERAL,P3
8,i m having an issue with the product purchased...,GENERAL,P3
9,my product purchased is making strange noises ...,GENERAL,P3


Step 4: Calculate SLA and due time

In [31]:

time_candidates = ['created_at','created','timestamp','time','created date','created_time']
created_col = next((c for c in time_candidates if c in df.columns), None)


if created_col is None:
    df['created_at'] = pd.Timestamp.utcnow()
    created_col = 'created_at'


df[created_col] = pd.to_datetime(df[created_col], errors='coerce', utc=True)


SLA_HOURS = {'P0': 2, 'P1': 4, 'P2': 24, 'P3': 72}

def compute_due_time(row):
    created = row[created_col]
    pr = row['priority']
    if pd.isna(created):
        return pd.NaT
    return created + pd.Timedelta(hours=SLA_HOURS.get(pr, 72))

df['due_time'] = df.apply(compute_due_time, axis=1)

df[[created_col, 'priority', 'due_time']].head(5)

,created_at,priority,due_time
0,2025-12-23 06:15:51.954724+00:00,P1,2025-12-23 10:15:51.954724+00:00
1,2025-12-23 06:15:51.954724+00:00,P3,2025-12-26 06:15:51.954724+00:00
2,2025-12-23 06:15:51.954724+00:00,P3,2025-12-26 06:15:51.954724+00:00
3,2025-12-23 06:15:51.954724+00:00,P3,2025-12-26 06:15:51.954724+00:00
4,2025-12-23 06:15:51.954724+00:00,P2,2025-12-24 06:15:51.954724+00:00


Step 5: Manager report and export

In [30]:
issue_counts = df['issue_type'].value_counts()
priority_counts = df['priority'].value_counts()

report = {
    'total_tickets': int(len(df)),
    'by_issue': issue_counts.to_dict(),
    'by_priority': priority_counts.to_dict()
}
report


output_csv = 'triage_output.csv'
df.to_csv(output_csv, index=False)
print("Saved:", output_csv)

Saved: triage_output.csv


Step 6: Validation checks and sample review

In [32]:

print("Null clean_message:", int(df['clean_message'].eq("").sum()))
print("Any NaT in due_time:", df['due_time'].isna().sum())


samples = []
for p in ['P0','P1','P2','P3']:
    s = df[df['priority']==p][['clean_message','issue_type','priority']].head(3)
    samples.append((p, s))
for p, s in samples:
    print(f"\nPriority {p} samples:")
    display(s)


defaulted = df[(df['issue_type']=='GENERAL') & (df['priority']=='P3')][['clean_message']].head(10)
print("\nGeneral/P3 examples:")
display(defaulted)

Null clean_message: 0
Any NaT in due_time: 0

Priority P0 samples:


,clean_message,issue_type,priority
2809,i m having an issue with the product purchased...,REFUND,P0
4457,i ve noticed a software bug in the product pur...,BUG,P0
5046,i m having an issue with the product purchased...,BUG,P0



Priority P1 samples:


,clean_message,issue_type,priority
0,i m having an issue with the product purchased...,PAYMENT,P1
6,i m unable to access my product purchased acco...,LOGIN,P1
12,i m having an issue with the product purchased...,PAYMENT,P1



Priority P2 samples:


,clean_message,issue_type,priority
4,i m having an issue with the product purchased...,DELIVERY,P2
82,i m having an issue with the product purchased...,DELIVERY,P2
178,i m having an issue with the product purchased...,DELIVERY,P2



Priority P3 samples:


,clean_message,issue_type,priority
1,i m having an issue with the product purchased...,GENERAL,P3
2,i m facing a problem with my product purchased...,GENERAL,P3
3,i m having an issue with the product purchased...,GENERAL,P3



General/P3 examples:


,clean_message
1,i m having an issue with the product purchased...
2,i m facing a problem with my product purchased...
3,i m having an issue with the product purchased...
5,i m facing a problem with my product purchased...
7,i m having an issue with the product purchased...
8,i m having an issue with the product purchased...
9,my product purchased is making strange noises ...
10,i m having an issue with the product purchased...
11,i m having an issue with the product purchased...
13,i ve recently set up my product purchased but ...


Final Download Triage_output.csv

In [49]:
from google.colab import files
files.download('triage_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Issue Breakdown



In [42]:
df['issue_type'].value_counts()

,count
issue_type,
GENERAL,6484
BUG,982
LOGIN,450
PAYMENT,254
REFUND,188
DELIVERY,111


Priority Breakdown


In [43]:
df['priority'].value_counts()

,count
priority,
P3,6649
P1,1709
P2,105
P0,6


 Sample Insights

In [48]:
df[['clean_message', 'issue_type', 'priority']].sample(100)

,clean_message,issue_type,priority
7515,i m having an issue with the product purchased...,GENERAL,P3
7056,i m having an issue with the product purchased...,GENERAL,P3
8073,i m having an issue with the product purchased...,GENERAL,P3
5955,i m having an issue with the product purchased...,GENERAL,P3
5355,i m facing issues logging into my product purc...,LOGIN,P3
...,...,...,...
423,i m having an issue with the product purchased...,GENERAL,P3
4721,i m having an issue with the product purchased...,GENERAL,P3
2316,i m having an issue with the product purchased...,LOGIN,P1
1319,i m having an issue with the product purchased...,GENERAL,P3
